# 2. End-to-End Machine Learning Project

In this Chapter, we will go through a fictitious Machine Learning project on real estate.

We will go through the following steps:

1. Look at the big picture
2. Get the data
3. Discover and visualize the data to gain insights
4. Prepare the data for Machine Learning algorithms
5. Select a model and train it
6. Fine-tune our model
7. Present our solution
8. Launch, monitor, and maintain our system

### 1. Look at the big picture

Three main things to do:

1. **Frame the problem**：what is the final objective?
2. **Select a performance measure**: how do we know how well is our algorithm performing? 
3. **Check the assumptions**: what did we assume so far? 

### 2. Get the data

First thing to do is to create the **workspace**. 
Another important step is creating an isolated environment so we can work on different projects without having conflicting library versions. 

Now we can start working with our Housing dataset. Let's download it using a short script:

In [1]:
import os
import tarfile
from six.moves import urllib

In [2]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

In [3]:
def fetch_housing_data(housing_url = HOUSING_URL, housing_path = HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path = housing_path)
    housing_tgz.close()

In [4]:
# download the data
fetch_housing_data()

URLError: <urlopen error [WinError 10054] 远程主机强迫关闭了一个现有的连接。>

In [ ]:
# load data using pandas
import pandas as pd

def load_housing_data(housing_path = HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
# check the data structure
housing = load_housing_data()
housing.head()

In [ ]:
# examine attribute types and number of non-null values
housing.info()

Note how `total_bedrooms` has only 20,433 non-null values, compared to 20,640 for the rest.

Let's inspect `ocean_proximity`, the only category which is not a numberical attribute (`float64` data type):

In [ ]:
housing["ocean_proximity"].value_counts()

For numberical attributes, we can use the `describe()` method:

In [ ]:
housing.describe()

Another quick way to get a feel of the type of data you are dealing with is to plot a **histogram** for each numerical attribute.

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt

housing.hist(bins=50, figsize=(20,15))
plt.show()

#### Creating a Test Set

It is good practice to set aside a portion of the dataset as test set as soon as possible (and not look at it ever again!).  
Typically we can do this by setting aside 20% of the original dataset: 

In [ ]:
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

However, this has a problem: everytime we run the program again it will generate a different test set so over time, you will go through the whole dataset again. Not good. 

Possible solutions:
1. Save the test set on the first run and then load it in subsequent runs
2. Set the random number generator’s seed (e.g.,  `np.ran dom.seed(42)`) always to same value before calling `np.random.permutation()` 

**Note**: clearly, my seed was chosen very very randomly here. 

However, both these may break with updated datasets. A common solution is to use each instance’s identifier to decide whether or not it should go in the test set (_assuming instances have a unique and immutable identifier_).

In [ ]:
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

Unfortunately, our housing dataset does not have an identifier column. We could use the row index as id: 

In [ ]:
import numpy as np

housing_with_id = housing.reset_index() # adds an `index` column
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

However, this creates contraints: we can only append at the end of the dataset, and no row can be deleted.  
Aleternatively, we could use latitute and longitude:

In [ ]:
housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id")

Next, to ensure **stratified sampling**, we may want to create an income category attribute with 5 categories:

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"],
                                bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                                labels=[1, 2, 3, 4, 5])

We can now to stratified sampling using Scikit-learn's `StratifiedShuffleSplit` class:

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

Let's see the proportion that went in different bins:

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

Let's remove the `income_cat` attribute so the data is back to its original state:

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

### 3. Discover and visualize the data to gain insights

1. We are only exploring the **training set**
2. If dataset is to big, it's good to have an **exploration set** for ease of manipulation
3. Always make a copy so you can mess around without worries

In [ ]:
housing = strat_train_set.copy()

### Visualizing geographical data

In [ ]:
housing.plot(kind = "scatter", x = "longitude", y = "latitude")

Let's diminuish alpha so we can see where there is a higher density of points:

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

Now let’s look at the housing prices, with circle radius representing population and color representing price:

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=housing["population"]/100, label="population", figsize=(10,7),
    c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
)
plt.legend()

### Looking for correlations

Using the Pearson's coefficient to measure **linear correlation**:

In [ ]:
corr_matrix = housing.corr()

In [ ]:
# let's look at median_house_value as an example
corr_matrix["median_house_value"].sort_values(ascending=False)

Another method involves Pandas' `scatter_matrix` function:

In [ ]:
# only focusing on a 4 x 4 subset of all the possible 11 x 11 plots
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
            "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

Let's look in more detail into the variable with the highest predictive power `median_income` using a scatter plot:

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value",
            alpha=0.1)

Our 500,000 dollars ceiling is clearly visible, and there are a few other horizontal lines (e.g. around 350,000) worth investigating further.

### Experimenting with attribute combinations

Sometimes default "pure" attributes are not the most relevant. For example, it may be useful to know the number of rooms per household rather than the two attributes independently. 
We can construct these new attributes by ourselves:

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

### 4. Prepare the data

There are a few arguments for doing this programmatically rather than manually:
1. Reproduce these transformations on any dataset
2. Build a library of reusable transformation functions
3. Use these functions in your live system to transform data before feeding it to algorithms
4. Easily try various transformations and see which combination of transformations works best

In [ ]:
# separating predictors and target values

housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

#### Data cleaning

Fist, let's take care of missing values for `total_bedrooms`. We have 3 options here:
1. Get rid of corresponding districts
2. Get rid of whole attribute
3. Fill in the missing values

In code, these options look like this:

In [ ]:
housing.dropna(subset=["total_bedrooms"])   # option 1
housing.drop("total_bedrooms", axis=1)      # option 2
median = housing["total_bedrooms"].median() # option 3 with median
housing["total_bedrooms"].fillna(median, inplace=True)

In Scikit-Learn there is a handy class for missing values: `SimpleImputer`:

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

Since this will only work on numerical values, we need to create a copy without the text attribute `ocean_proximity`:

In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)

Now we can fit the imputer instance to the training data:

In [ ]:
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
# checking that the median has been computed correctly
housing_num.median().values

Now we can transform the training set by replacing missing values by the learned medians:

In [ ]:
X = imputer.transform(housing_num)

**Note**: we could do these two steps (`fit` and `transform`) much faster using `fit_transform`. 

### Handling text and categorical attributes

In order to be able to work with text attributes, we could convert them from text to numbers using the `OrdinalEncoder` class:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

housing_cat = housing[["ocean_proximity"]]
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

# check
housing_cat_encoded[:10]

One issue with this representation is that ML algorithms will assume that two nearby values are more similar than two distant values. This may be true sometimes but definitely not in this case.  
To fix this issue, a common solution is to create a binary attribute (*dummy variables*) in a process known as *one-hot encoding*. 

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

**Note** SciPy sparse matrix means that instead of storing all the 0s and 1s, we only store the location of 1s.  

We can get back to dense Numpy arrays like this:

In [ ]:
housing_cat_1hot.toarray()

#### Custom transformers

To be compatible with Scikit-learn, you need to create a class and implement three methods: `fit()`, `tranform()` and `fit_transform`.  

We can get the last one by simply adding `TransformerMixin` as a base class. Also, if we add `BaseEstimator` as a base class and avoid `*args` and `**kargs` we will get two extra methods (`get_params()` and `set_params()`). 

Example:

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): 
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self 
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room: # conditional to hyperparameter
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                        bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In this example we have one hyperparameter `add_bedrooms_per_room` set on `True` by default.  

Setting hyperparameter is a good way to simplify tuning in the future and automate data preparation steps.

#### Feature scaling

Machine Learning algorithms usually don't perform very well on attributes that have very different scales. We have to options to solve this:

1. **Min-max scaling / Normalization** (`MinMaxScaler`) returns number in range [0,1] = $\displaystyle \frac{value - min}{max - min}$  
2. **Standardization** (`StandardScaler`) does not return value in specific range = $\displaystyle \frac{value - \mu}{\sigma}$ 

**Note**: since it does not depend on single values (max and min), standardization is less affected by outliers. 

#### Transformation pipelines

To help with pipelines (sequences of transformations), Scikit-Learn provides the (.. drum roll ..) `Pipeline` class. Here is an example for numerical attributes:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

It would be more convenient to have a single transformer able to handle all columns, applying the appropriate transformations to each column.  

This is exactly what `ColumnTransformer` does, and it works very well with Pandas dataframes:

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])

housing_prepared = full_pipeline.fit_transform(housing)

**Note 1**: In order to decide if returning a sparse or dense matrix, we can set a threshold value so that if density < threshold we will go for a sparse matrix. 
The default is set to **0.3**.  

**Note 2**: We can also `drop` the columns or `pass through` them, doing nothing. 

**Note 3**: By default, columns not explicitely mentioned will be dropped. You can set the `remainder` parameter to handle these cols differently. 

### 5. Select and train a model

#### Training and Evaluating on the Training Set

Let's first train a simple Linear Regression model:

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

Let's try our model on a few instances:

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", lin_reg.predict(some_data_prepared))

In [ ]:
print("Labels:", list(some_labels))

It works, although it is clearly not very accurate. Let's check the root mean square error (RMSE) for the whole dataset:

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

Quite high. This could be due to the quality of the data we have or to the level of sophistication in our model. 

Let's try a more complex model to see how things change:

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
# evaluation of training set

housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

Perfect model?! No. Probably a bad case of overfitting. 

#### Better Evaluation Using Cross-Validation

One way to evaluate the Decision Tree model would be to use the `train_test_split` function to split the training set into a smaller training set and a validation set, then train your models against the smaller training set and evaluate them against the validation set.

An alternative is using Scikit-Learn k-fold cross-validation feature:

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                        scoring = "neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(tree_rmse_scores)

Wow, it managed to do worse than the linear regression model. Just to be sure, let's compute the scores for the Linear Regression as well:

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, 
                            scoring="neg_mean_squared_error", cv=10)

lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Yep, our Decision Trees model overfits the data so badly to underperform our linear regression model. 

Third model: Random Forests.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                        scoring = "neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)

display_scores(forest_rmse_scores)

On the other hand, Random Forests looks quite promising, albeit probably still overfitting. Possible solutions are regularization and collecting more training data.

**Best practice**: save each model (with hyperparameters, trained parameters, cross-validation scores and actual predictions) to easily compare different models.

In [ ]:
from sklearn.externals import joblib

joblib.dump(forest_reg, "forest_reg.pkl")

# and later...
my_model_loaded = joblib.load("forest_reg.pkl")

### 6. Fine-tune our model

#### Grid search

`GridsearchCV` is great to automate hyperparameters tuning. Input hyperparameters and values to try out, and it will evaluate all the possible combinations of hyperparameter values, using cross-validation. 

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}, 
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,  # five-fold cross validation
                            scoring='neg_mean_squared_error',
                            return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
# best combination of parameters
grid_search.best_params_

In [ ]:
# best estimator
grid_search.best_estimator_

**Note**: If  GridSearchCV is initialized with `refit=True` (which is the default), then once it finds the best estimator using cross-validation, it retrains it on the whole training set. This is usually a good idea since feeding it more data will likely improve its performance.

In [ ]:
# evaluation scores
cvres = grid_search.cv_results_

In [ ]:
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

#### Randomized search

When the hyperparameter search space is large, it is often preferable to use `RandomizedSearchCV`, which evaluates a given number of random combinations by selecting a random value for each hyperparameter at every iteration.

#### Ensemble model

We can also combine the best performing models, especially if the individual models make very different types of errors.

#### Analyze the best models and their errors

By looking more closely at our best models we can probably understand more about the relative importance of each attribute:

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_

In [ ]:
feature_importances

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

#### Evaluate your system on the test set

Now is the time to evaluate the final model on the test set:

In [ ]:
final_model = grid_search.best_estimator_
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()
X_test_prepared = full_pipeline.transform(X_test) # NOT fit.transform
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

Instead of a point estimate, it may be more helpful to have a confidence interval:

In [ ]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                        loc=squared_errors.mean(),
                        scale=stats.sem(squared_errors)))

### 7. Present your solution

The book doesn't cover this section extensively, it just mentions to: 
1. Present our solution (what we have learned, what worked / what did not, assumptions, limitations)
2. Dcument everything
3. Create presentations with clear visualizations and easy-to-remember statements

### 8. Launch, Monitor, and Maintain Your System

Ready and approved for production. Next steps:

1. Plug the production input data sources into your system
2. Write tests
3. Write monitoring code to check system’s live performance at regular intervals and trigger alerts when it drops
4. Monitor inputs (especially true for online learning systems)

**Note**: checking performance is important not only for sudden breakage, but also to monitor model degradation. 

### **Exercises**

### Ex1

Try a Support Vector Machine regressor (`sklearn.svm.SVR`), with various hyperparameters such as `kernel="linear"` (with various values for the `C` hyperparameter) or `kernel="rbf"` (with various values for the `C` and `gamma` hyperparameters). 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

param_grid = [
    {'kernel': ['linear'], 'C': [10 , 20, 50, 100]},
    {'kernel': ['rbf'], 'C': [10 , 20, 50, 100], 
     'gamma': [10 , 20, 50, 100]}, 
]

svm_reg = svm.SVR()

grid_search = GridSearchCV(svm_reg, param_grid, cv=5,  
                            scoring='neg_mean_squared_error',
                            return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
neg_mse = grid_search.best_score_
rmse = np.sqrt(-neg_mse)
rmse

### Ex2

Try replacing `GridSearchCV` with `RandomizedSearchCV`.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal

par_dist = [
    {'kernel': ['linear', 'rbf'],
     'C' : reciprocal(10,10000),
     'gamma': expon(scale=1), 
    }

svm_reg = svm.SVR()

grid_rand_search = RandomizedSearchCV(svm_reg, param_distributions=par_dist, n_iter=20,
                                      cv = 5, scoring='neg_mean_squared_error',
                                      return_train_score=True)

grid_rand_search.fit(housing_prepared, housing_labels)

In [ ]:
neg_mse = grid_rand_search.best_score_
rand_rmse = np.sqrt(-neg_mse)
rand_rmse

### Ex3

Try adding a transformer in the preparation pipeline to select only the most important attributes.

> TO DO

### Ex4

Try creating a single pipeline that does the full data preparation plus the final prediction.

In [11]:
from sklearn.pipeline import Pipeline

prep_pred_pipeline = Pipeline([
    ('prep', full_pipeline), # reusing our prep pipeline from section 4 above
    ('pred', rnd_search.predict(data, labels)) # using randomized search from ex. 2
])

NameError: name 'full_pipeline' is not defined

In [12]:
prep_pred_pipeline.fit(housing, housing_labels)

NameError: name 'prep_pred_pipeline' is not defined

### Ex5

Automatically explore some preparation options using `GridSearchCV`.

My solution: trying to find the best imputer between mean, median and most frequent.

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

params = {[
    'imputer': ['mean', 'median', 'most frequent']
]}

grid_search_prep = GridSearchCV(prep_pred_pipeline, params, cv = 5, scoring = 'neg_mean_squared_error')
grid_search_prep.fit(housing, housing_labels)

SyntaxError: invalid syntax (<ipython-input-13-9a6505a0fe95>, line 5)

In [14]:
grid_search_prep.best_params_

NameError: name 'grid_search_prep' is not defined

**Note**: issues loading the housing dataset. Need to come back to this later and check it with actual data.  